In [1]:
import sys

sys.path.append("../")

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
from positioning.readwav import readwav
from positioning.get_spectrum_amplitude import (
    get_tukey_spectrum_amplitude,
    get_sn_amplitude,
)
from positioning.sound_db import ReflectCeilingDB
from positioning import tools

In [3]:
plt.style.use(["science", "notebook"])

In [4]:
signal = readwav("../data/pos-reflect-ceiling/p28.wav")[:, 1]
signal.shape

(10331520,)

In [5]:
import scipy.signal as sg
from positioning.make_wave import reference_transmit_signal, reference_transmit_tukey

In [6]:
sig = readwav("../data/reference_3d_phone/a0e0.wav")[:, 1]

In [7]:
res_signal = signal
first_freq: int = 15000
last_freq: int = 22000
interval_length: float = 0.2
sampling_rate = 48000  # マイクのサンプリングレート
signal_length = 0.003  # チャープ一発の信号長
interval_sample_length = int(interval_length * sampling_rate)  # チャープのバンド間の間隔のサンプル数
len_chirp_sample = 144  # 受信したチャープのサンプル数
chirp_width = 1000  # チャープ一発の周波数帯域の幅
band_freqs = np.arange(first_freq, last_freq, chirp_width)  # 送信する周波数のバンド
sampling_buffer = 48  # データ切り出しの前後のゆとりN_c (1ms)
fft_freq_rate = sampling_rate / len_chirp_sample  # FFTの周波数分解能
band_freq_index_range = int(chirp_width / fft_freq_rate + 1)  # 1つの帯域の周波数インデックスの範囲
tukey = sg.windows.tukey(int(sampling_rate * 0.003))
# マッチドフィルター
sample_frame_length = int(interval_length * 20 * sampling_rate)  # 0.1秒分のサンプル数
chirp = reference_transmit_tukey(
    first_freq=first_freq, last_freq=last_freq, interval_length=interval_length
)  # 参照信号の生成
corr = sg.correlate(res_signal[:sample_frame_length], chirp, mode="valid")  # 相互相関
corr_lags = sg.correlation_lags(
    len(res_signal[:sample_frame_length]), len(chirp), mode="valid"
)
index_f = corr_lags[np.abs(corr).argmax()]  # 最大値のインデックス見つける

In [8]:
%matplotlib qt6

In [9]:
sns.lineplot(x=corr_lags, y=np.abs(corr))

<Axes: >

In [10]:
corr

array([-1.35015131e-04-9.66007382e-05j,  7.68194131e-05+1.53817071e-04j,
        1.34151218e-05-1.70775298e-04j, ...,
        4.19151650e-04-1.90469090e-04j, -3.49380302e-04-1.36062796e-04j,
        9.88236780e-05+2.40134342e-04j])

In [11]:
# plt.plot(res_signal[:sample_frame_length])

In [12]:
# plt.plot(chirp)

In [13]:
argrelmax = sg.argrelmax(np.abs(corr), order=40)
print(argrelmax)

(array([    41,    106,    187, ..., 133198, 133262, 133350]),)


In [14]:
print(index_f)

61257


In [15]:
test_pos = np.array([0.25, 1.25, 1])
real_spk_pos = np.array([0, 0, 2.2])
virtual_spk_pos = np.array([0, 0, 2.8])
real_distance = np.linalg.norm(test_pos - real_spk_pos)
virtual_distance = np.linalg.norm(test_pos - virtual_spk_pos)
print(real_distance)
print(virtual_distance)

1.7507141400011597
2.2056745000112774


In [16]:
distance_diff = np.abs(virtual_distance - real_distance)
diff_sample = int(distance_diff / 340 * sampling_rate)
print(diff_sample)

64


In [17]:
import positioning.tools as tools

In [18]:
r, theta, phi = tools.rect_to_polar_3d(*(test_pos - real_spk_pos))
print(r)
print(theta)
print(phi)

1.7507141400011597
11.309932474020213
43.269793486721156


In [19]:
from positioning.create_db import create_3d_spectrum_db
from positioning.estimate import estimate_direction_3d, estimate_height, calc_position

In [20]:
db = create_3d_spectrum_db("../data/reference_3d_phone")

In [21]:
direction = estimate_direction_3d(db, res_signal)

In [22]:
print(direction)

[-28   0]


In [23]:
true_pos = pd.read_csv("../data/pos-reflect-ceiling/measure-points.csv", index_col=0)
true_pos.head()

,x,y,z
p28,-0.25,1.25,1.00
p29,0.25,1.25,1.00
p30,-0.25,1.75,1.00
p31,0.25,1.75,1.00
p32,-0.25,1.25,1.25


In [24]:
r, theta, phi = tools.rect_to_polar_3d(
    true_pos["x"], true_pos["y"], true_pos["z"] - 2.2
)

In [25]:
true_pos["distance"] = r
true_pos["azimuth"] = theta
true_pos["elevation"] = phi
true_pos.head()

,x,y,z,distance,azimuth,elevation
p28,-0.25,1.25,1.00,1.750714,-11.309932,43.269793
p29,0.25,1.25,1.00,1.750714,11.309932,43.269793
p30,-0.25,1.75,1.00,2.136586,-8.130102,34.169544
p31,0.25,1.75,1.00,2.136586,8.130102,34.169544
p32,-0.25,1.25,1.25,1.589811,-11.309932,36.695001


In [26]:
true_pos

,x,y,z,distance,azimuth,elevation
p28,-0.25,1.25,1.00,1.750714,-11.309932,43.269793
p29,0.25,1.25,1.00,1.750714,11.309932,43.269793
p30,-0.25,1.75,1.00,2.136586,-8.130102,34.169544
p31,0.25,1.75,1.00,2.136586,8.130102,34.169544
p32,-0.25,1.25,1.25,1.589811,-11.309932,36.695001
p33,0.25,1.25,1.25,1.589811,11.309932,36.695001
p34,-0.25,1.75,1.25,2.006863,-8.130102,28.253635
p35,0.25,1.75,1.25,2.006863,8.130102,28.253635
p36,-0.25,1.25,0.75,1.930673,-11.309932,48.679962
p37,0.25,1.25,0.75,1.930673,11.309932,48.679962


In [27]:
cur_true_pos = true_pos.loc[["p34", "p35"]]
test_signal = [
    readwav(f"../data/pos-reflect-ceiling/{i}.wav")[:, 1] for i in cur_true_pos.index
]

In [29]:
est_direction = np.array(
    [
        [estimate_direction_3d(db, s[i * 96000 : (i + 2) * 96000]) for i in range(90)]
        for s in test_signal
    ]
)

In [30]:
print(est_direction)

[[[ 12   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 12   2]
  [ 12   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 12   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 12   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 19  27]
  [ 13   2]
  [ 20  27]
  [ 20  27]
  [-34  21]
  [ 19  27]
  [ 19  27]
  [ 20  27]
  [ 20  27]
  [ 13   2]
  [ 23  27]
  [ 13   2]
  [ 20  27]
  [ 20  27]
  [-35  21]
  [-35  21]
  [ 19  27]
  [ 19  27]
  [ 13   2]
  [ 13   2]
  [ 19  27]
  [ 19  27]
  [ 13   2]
  [ 19  27]
  [ 23  27]
  [ 13   2]
  [ 23  27]
  [ 23  27]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 12   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 13   3]
  [ 13   2]
  [ 22  28]
  [ 13   2]
  [ 13   2]
  [ 13   2]
  [ 

In [32]:
azimuth_error = np.array(
    [
        np.abs(est_direction[i, :, 0] - az)
        for i, az in enumerate(cur_true_pos["azimuth"])
    ]
)
print(azimuth_error)

[[20.13010235 20.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  21.13010235 20.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  21.13010235 21.13010235 20.13010235 20.13010235 20.13010235 21.13010235
  21.13010235 21.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  20.13010235 20.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  21.13010235 21.13010235 21.13010235 20.13010235 20.13010235 21.13010235
  21.13010235 21.13010235 21.13010235 21.13010235 27.13010235 21.13010235
  28.13010235 28.13010235 25.86989765 27.13010235 27.13010235 28.13010235
  28.13010235 21.13010235 31.13010235 21.13010235 28.13010235 28.13010235
  26.86989765 26.86989765 27.13010235 27.13010235 21.13010235 21.13010235
  27.13010235 27.13010235 21.13010235 27.13010235 31.13010235 21.13010235
  31.13010235 31.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  20.13010235 21.13010235 21.13010235 21.13010235 21.13010235 21.13010235
  21.13010235 30.13010235 21.13010235 

In [34]:
elevation_error = np.array(
    [
        np.abs(est_direction[i, :, 1] - el)
        for i, el in enumerate(cur_true_pos["elevation"])
    ]
)
print(elevation_error)

[[26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542  1.25363542 26.25363542
   1.25363542  1.25363542  7.25363542  1.25363542  1.25363542  1.25363542
   1.25363542 26.25363542  1.25363542 26.25363542  1.25363542  1.25363542
   7.25363542  7.25363542  1.25363542  1.25363542 26.25363542 26.25363542
   1.25363542  1.25363542 26.25363542  1.25363542  1.25363542 26.25363542
   1.25363542  1.25363542 26.25363542 26.25363542 26.25363542 26.25363542
  26.25363542 26.25363542 26.25363542 26.25363542 26.25363542 25.25363542
  26.25363542  0.25363542 26.25363542 

In [35]:
df_est = pd.DataFrame()
for i, d in enumerate(est_direction):
    df_est = pd.concat(
        [df_est, pd.DataFrame(d, columns=["azimuth", "elevation"]).assign(test=i)]
    ).reset_index(drop=True)
df_est["azimuth_error"] = azimuth_error.reshape(-1)
df_est["elevation_error"] = elevation_error.reshape(-1)
df_est.head()

,azimuth,elevation,test,azimuth_error,elevation_error
0,12,2,0,20.130102,26.253635
1,12,2,0,20.130102,26.253635
2,13,2,0,21.130102,26.253635
3,13,2,0,21.130102,26.253635
4,13,2,0,21.130102,26.253635


In [36]:
sns.displot(df_est, x="azimuth_error", kind="ecdf")
plt.xlabel("Azimuth error [deg]")
plt.ylabel("CDF")

Text(9.444444444444445, 0.5, 'CDF')

In [37]:
sns.displot(df_est, x="elevation_error", kind="ecdf")
plt.xlabel("Elevation error [deg]")
plt.ylabel("CDF")

Text(9.444444444444445, 0.5, 'CDF')

In [38]:
v_distance, _, _ = tools.rect_to_polar_3d(
    true_pos["x"], true_pos["y"], true_pos["z"] - 2.8
)
v_distance

p28    2.205675
p29    2.205675
p30    2.522895
p31    2.522895
p32    2.006863
p33    2.006863
p34    2.351064
p35    2.351064
p36    2.414022
p37    2.414022
p38    2.706936
p39    2.706936
dtype: float64

In [39]:
true_pos["v_distance"] = v_distance

In [40]:
true_pos["distance_diff"] = np.abs(true_pos["distance"] - true_pos["v_distance"])

In [44]:
true_pos.head()

,x,y,z,distance,azimuth,elevation,v_distance,distance_diff
p28,-0.25,1.25,1.00,1.750714,-11.309932,43.269793,2.205675,0.454960
p29,0.25,1.25,1.00,1.750714,11.309932,43.269793,2.205675,0.454960
p30,-0.25,1.75,1.00,2.136586,-8.130102,34.169544,2.522895,0.386309
p31,0.25,1.75,1.00,2.136586,8.130102,34.169544,2.522895,0.386309
p32,-0.25,1.25,1.25,1.589811,-11.309932,36.695001,2.006863,0.417052


In [47]:
true_pos["sample_diff"] = (true_pos["distance_diff"] / 340 * sampling_rate).astype(int)

In [48]:
true_pos.head()

,x,y,z,distance,azimuth,elevation,v_distance,distance_diff,sample_diff
p28,-0.25,1.25,1.00,1.750714,-11.309932,43.269793,2.205675,0.454960,64
p29,0.25,1.25,1.00,1.750714,11.309932,43.269793,2.205675,0.454960,64
p30,-0.25,1.75,1.00,2.136586,-8.130102,34.169544,2.522895,0.386309,54
p31,0.25,1.75,1.00,2.136586,8.130102,34.169544,2.522895,0.386309,54
p32,-0.25,1.25,1.25,1.589811,-11.309932,36.695001,2.006863,0.417052,58


In [49]:
true_pos

,x,y,z,distance,azimuth,elevation,v_distance,distance_diff,sample_diff
p28,-0.25,1.25,1.00,1.750714,-11.309932,43.269793,2.205675,0.454960,64
p29,0.25,1.25,1.00,1.750714,11.309932,43.269793,2.205675,0.454960,64
p30,-0.25,1.75,1.00,2.136586,-8.130102,34.169544,2.522895,0.386309,54
p31,0.25,1.75,1.00,2.136586,8.130102,34.169544,2.522895,0.386309,54
p32,-0.25,1.25,1.25,1.589811,-11.309932,36.695001,2.006863,0.417052,58
p33,0.25,1.25,1.25,1.589811,11.309932,36.695001,2.006863,0.417052,58
p34,-0.25,1.75,1.25,2.006863,-8.130102,28.253635,2.351064,0.344200,48
p35,0.25,1.75,1.25,2.006863,8.130102,28.253635,2.351064,0.344200,48
p36,-0.25,1.25,0.75,1.930673,-11.309932,48.679962,2.414022,0.483348,68
p37,0.25,1.25,0.75,1.930673,11.309932,48.679962,2.414022,0.483348,68
